# Load Data

In [39]:
%reload_ext autoreload
%autoreload 2

In [40]:
from result_saver import SaverProvider
provider = SaverProvider()

In [41]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
243,2023-10-30 09:43:24.564124+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsb5daqbg008sjvb0,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,[],10000.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...


In [64]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories["mmr_log_0"][:1000]

# UF Decoder

In [66]:
import pickle
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout, get_KDEs, get_counts

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_nazca"), _is_hex=True)
kde_dict, scaler_dict = get_KDEs(provider, DEVICE, layout, bandwidths=0.2, plot=False)
code = RepetitionCodeCircuit(max_distance, max_distance)


In [67]:
from qiskit_qec.decoders.hdrg_decoders import UnionFindDecoder

dcd_pkl_str = f"dist_{max_distance}_decoder.pkl"

try:
    with open(f"dist_{max_distance}_decoder.pkl", 'rb') as f:
        decoder = pickle.load(f)

except FileNotFoundError:
    print("decoder not found, creating a new one...")
    code = RepetitionCodeCircuit(max_distance, max_distance)
    decoder = UnionFindDecoder(code)    

    print("saving the decoder...")
    with open(dcd_pkl_str, 'wb') as f:
        pickle.dump(decoder, f)

In [68]:
from soft_info import soft_reweight, get_counts, rx_draw_2D
from tqdm import tqdm

VERBOSE = True

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))):
    
    with open(dcd_pkl_str, 'rb') as f: # Reload the decoder each time to reset the graph
        decoder = pickle.load(f)

    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    
    if VERBOSE and logical[0] == 1:
        print("\nmeas_str:", count_key)
        print("flipped_qubit_dict", flipped_qubit_dict)
        print("logical:", logical, "\n")
            
    if logical[0] == 1:
            print("------------ Wrong logical! ------------\n")
    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


  0%|          | 4/1000 [00:00<01:19, 12.55it/s]


meas_str: 0111001110 000001000 010100001 010001010 010000010 010000000 100000000 010000000 000000000 000000000 000000000
flipped_qubit_dict {'Cluster_0': [8], 'Cluster_1': [8], 'Cluster_2': [1, 0, 1], 'Cluster_3': [8, 8, 9, 2, 2, 4, 5]}
logical: [1] 

------------ Wrong logical! ------------



 10%|█         | 100/1000 [00:08<01:17, 11.56it/s]


meas_str: 0010011000 011111000 001101100 001101000 001100100 001101011 001001101 001101011 000001000 001101000 000000000
flipped_qubit_dict {'Cluster_0': [], 'Cluster_1': [2], 'Cluster_2': [8, 4, 9, 6, 4, 7, 5, 4, 5, 4, 6, 5, 6], 'Cluster_3': [0, 1], 'Cluster_4': [7]}
logical: [1] 

------------ Wrong logical! ------------



 11%|█         | 110/1000 [00:09<01:16, 11.66it/s]

...


meas_str: 0001110110 110011001 111010101 110011011 111011101 000000001 111011111 110000011 110000111 000000000 000000110
flipped_qubit_dict {'Cluster_0': [2], 'Cluster_1': [8], 'Cluster_2': [8], 'Cluster_3': [4], 'Cluster_4': [7, 9], 'Cluster_5': [0, 0, 1, 1, 4, 1, 2, 0, 3, 1], 'Cluster_6': [8]}
logical: [1] 

------------ Wrong logical! ------------



 95%|█████████▍| 948/1000 [01:24<00:04, 10.59it/s]


meas_str: 1110101000 111111110 110000010 111111110 010000000 110000110 000000000 110000000 000000000 110000000 000000000
flipped_qubit_dict {'Cluster_0': [8], 'Cluster_1': [2], 'Cluster_2': [9, 8, 9], 'Cluster_3': [4], 'Cluster_4': [6], 'Cluster_5': [0, 1]}
logical: [1] 

------------ Wrong logical! ------------



100%|██████████| 1000/1000 [01:29<00:00, 11.21it/s]


Logical_counts: {'0': 984, '1': 16}


# PyMatching

In [35]:
import stim
import pymatching

d = max_distance
T = max_distance

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)


In [38]:
from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one

VERBOSE = False

actual_observables = np.array([[False]]) # hardcoded, can be retrieved
num_errors = 0

i = 0
w_idx_lst = []
for shot in tqdm(range(len(memory))):
    i += 1
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
    count_key = next(iter(counts.keys()))
    
    #soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
    reweight_edges_to_one(matching)

    array_processed_string = counts_to_det_syndr(count_key, verbose=False)

    predicted_observables = matching.decode(array_processed_string)

    if predicted_observables == [0]:
        continue
    
    #print(f"Wrong decoding at index {i}")
    w_idx_lst.append(i)

    
    if VERBOSE:
        print("Count key:", count_key)
        print("count_string_to_syndromes:", array_processed_string)

    if VERBOSE:
        draw_matching_graph(matching, d, T)

    if VERBOSE:
        matched_edges = matching.decode_to_edges_array(array_processed_string)
        print("matched_edges: ", matched_edges)
        print("Estimated flip:", predicted_observables)

    num_errors += not np.array_equal(actual_observables[0, :], predicted_observables) # 0 can be changed to i if multiple observables and multiple syndromes per ovbservable

print("Num errors:", num_errors)

100%|██████████| 100/100 [00:06<00:00, 14.54it/s]

Num errors: 22
